In [6]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import tensorboard
from tensorboardX import SummaryWriter
from PIL import Image
  

In [7]:
#data preperation

In [8]:
transform=transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize(
                                  mean=[0.5,0.5,0.5],
                                  std=[0.5,0.5,0.5]
                              )])

In [9]:
batch_size=4

trainset = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [10]:
#Build  the model

In [11]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.tanh(self.conv1(x)))
        x = self.pool(F.tanh(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)
for epoch in range(20):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

/opt/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:1698: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


[1,  2000] loss: 2.287
[1,  4000] loss: 2.187
[1,  6000] loss: 2.087
[1,  8000] loss: 2.030
[1, 10000] loss: 2.009
[1, 12000] loss: 1.976
[2,  2000] loss: 1.932
[2,  4000] loss: 1.899
[2,  6000] loss: 1.838
[2,  8000] loss: 1.801
[2, 10000] loss: 1.755
[2, 12000] loss: 1.723
[3,  2000] loss: 1.673
[3,  4000] loss: 1.642
[3,  6000] loss: 1.644
[3,  8000] loss: 1.644
[3, 10000] loss: 1.613
[3, 12000] loss: 1.603
[4,  2000] loss: 1.557
[4,  4000] loss: 1.557
[4,  6000] loss: 1.556
[4,  8000] loss: 1.546
[4, 10000] loss: 1.504
[4, 12000] loss: 1.516
[5,  2000] loss: 1.482
[5,  4000] loss: 1.461
[5,  6000] loss: 1.467
[5,  8000] loss: 1.449
[5, 10000] loss: 1.430
[5, 12000] loss: 1.420
[6,  2000] loss: 1.406
[6,  4000] loss: 1.391
[6,  6000] loss: 1.386
[6,  8000] loss: 1.369
[6, 10000] loss: 1.362
[6, 12000] loss: 1.354
[7,  2000] loss: 1.342
[7,  4000] loss: 1.323
[7,  6000] loss: 1.328
[7,  8000] loss: 1.334
[7, 10000] loss: 1.314
[7, 12000] loss: 1.288
[8,  2000] loss: 1.296
[8,  4000] 

In [13]:
#test the model 
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 62 %


In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)
for epoch in range(20):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 0.996
[1,  4000] loss: 0.997
[1,  6000] loss: 1.004
[1,  8000] loss: 0.994
[1, 10000] loss: 0.972
[1, 12000] loss: 0.985
[2,  2000] loss: 0.943
[2,  4000] loss: 0.957
[2,  6000] loss: 0.980
[2,  8000] loss: 0.963
[2, 10000] loss: 0.965
[2, 12000] loss: 0.943
[3,  2000] loss: 0.915
[3,  4000] loss: 0.921
[3,  6000] loss: 0.926
[3,  8000] loss: 0.943
[3, 10000] loss: 0.920
[3, 12000] loss: 0.931
[4,  2000] loss: 0.884
[4,  4000] loss: 0.900
[4,  6000] loss: 0.890
[4,  8000] loss: 0.885
[4, 10000] loss: 0.909
[4, 12000] loss: 0.902
[5,  2000] loss: 0.867
[5,  4000] loss: 0.872
[5,  6000] loss: 0.859
[5,  8000] loss: 0.886
[5, 10000] loss: 0.865
[5, 12000] loss: 0.864
[6,  2000] loss: 0.817
[6,  4000] loss: 0.848
[6,  6000] loss: 0.826
[6,  8000] loss: 0.856
[6, 10000] loss: 0.863
[6, 12000] loss: 0.848
[7,  2000] loss: 0.805
[7,  4000] loss: 0.819
[7,  6000] loss: 0.844
[7,  8000] loss: 0.847
[7, 10000] loss: 0.796
[7, 12000] loss: 0.791
[8,  2000] loss: 0.791
[8,  4000] 

In [15]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 65 %
